# Working with timestamps and bursts

*This notebook is part of a [tutorial series](https://github.com/tritemio/FRETBursts_notebooks) for the [FRETBursts](http://tritemio.github.io/FRETBursts/) burst analysis software.*

> In this notebook show how to access different streams of timestamps,
> burst data (i.e. start and stop times and indexes, counts, etc...).
> This is useful for users wanting to access or process bursts data 
> and timestamps in custom ways.

# Load data

We start by loading the data, computing background and performing a standard burst search:

In [1]:
from fretbursts import *
sns = init_notebook()

 - Optimized (cython) burst search loaded.
 - Optimized (cython) photon counting loaded.
--------------------------------------------------------------
 You are running FRETBursts (version 0.5+3.g84a3c1b).

 If you use this software in a publication, please cite it as:

   FRETBursts - An opensource single-molecule FRET bursts analysis toolkit.
   A. Ingargiola 2014. http://tritemio.github.io/FRETBursts

--------------------------------------------------------------

/Users/anto/miniconda3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
filename = "./data/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5"
d = loader.photon_hdf5(filename)
loader.alex_apply_period(d)
d.calc_bg(bg.exp_fit, time_s=30, tail_min_us='auto', F_bg=1.7)
d.burst_search()

#donor: 721537  #acceptor: 1537985 

 - Calculating BG rates ... 

[DONE]


 - Performing burst search (verbose=False) ...

[DONE]


 - Calculating burst periods ...

...

 - Counting D and A ph and calculating FRET ... 


   - Applying background correction.


   - Applying leakage correction.


   - Applying direct excitation correction.


   [DONE Counting D/A]


# Getting the timestamps

To get the timestamps arrays for a given [photon stream](http://fretbursts.readthedocs.org/en/latest/ph_sel.html) we use [Data.get_ph_times](http://fretbursts.readthedocs.org/en/latest/data_class.html?highlight=get_ph_times#fretbursts.burstlib.Data.get_ph_times). Here a few example:

In [3]:
ph = d.get_ph_times()                              # all the recorded photons
ph_dd = d.get_ph_times(ph_sel=Ph_sel(Dex='Dem'))   # donor excitation, donor emission
ph_d = d.get_ph_times(ph_sel=Ph_sel(Dex='DAem'))   # donor excitation, donor+acceptor emission
ph_aa = d.get_ph_times(ph_sel=Ph_sel(Aex='Aem'))   # acceptor excitation, acceptor emission

This are streams of all timestamps (both inside and outside the bursts).
Similarly, we can get "masks" of photons for each photon stream using 
[Data.get_ph_mask](http://fretbursts.readthedocs.org/en/latest/data_class.html?highlight=get_ph_mask#fretbursts.burstlib.Data.get_ph_mask):

In [4]:
mask_dd = d.get_ph_mask(ph_sel=Ph_sel(Dex='Dem'))   # donor excitation, donor emission
mask_d = d.get_ph_mask(ph_sel=Ph_sel(Dex='DAem'))   # donor excitation, donor+acceptor emission
mask_aa = d.get_ph_mask(ph_sel=Ph_sel(Aex='Aem'))   # acceptor excitation, acceptor emission

Masks are arrays of booleans (True or False values) which are True
when the corresponding photon is in the stream. Note that all masks
have same number of elements as the all-photons timestamps array:

In [5]:
ph.size, mask_dd.size, mask_d.size, mask_aa.size

(2259522, 2259522, 2259522, 2259522)

Masks can be used to count photons in one stream:

In [6]:
mask_d.sum()

1434842

and to obtain the timestamps for one stream:

In [7]:
ph[mask_d]

array([     187345,      379190,      402492, ..., 47999838819,
       47999862958, 47999954653])

Note that the arrays `ph[mask_d]` and `ph_d` are equal. This is an important point to understand.

# Burst data

There are several fields containing burst data:

**Start-stop**:

- `Data.mburst`: start-stop information for each burst

**Counts**:
- `Data.nd`: donor detetor counts during donor excitation
- `Data.na`: acceptor detetor counts during donor excitation
- `Data.naa`: acceptor detetor counts during acceptor excitation (ALEX only)
- `Data.nda`: donor detetor counts during acceptor excitation

**FRET**:
- `Data.E`: Proximity Ratio (when uncorrected) or FRET efficiency (when corrected)
- `Data.S`: "Stoichiometry" (ALEX only)

All these fields are lists containing one element per each channel.
In single-spot data these list have one single element which is accessed
using the `[0]` notation:

In [8]:
bursts = d.mburst[0]
nd = d.nd[0]
na = d.na[0]
naa = d.naa[0]
E = d.E[0]
S = d.S[0]

## Burst start and stop

The start-stop burst data is in `bursts` (a variable of type [Bursts](), plural):

In [9]:
bursts

,istart,istop,start,stop
0,153,189,5020488,5159285
1,184,194,5136893,5186298
2,215,224,6186769,6247541
3,240,250,6487794,6548720
4,327,339,8695237,8781357
...,...,...,...,...
27585,2259224,2259233,47992532316,47992589568
27586,2259411,2259433,47997934377,47998022338
27587,2259442,2259451,47998171399,47998230689
27588,2259449,2259459,47998210710,47998281141


Indexing `bursts` we can access a single burst:

In [10]:
firstburst = bursts[0]
firstburst

,istart,istop,start,stop
0,153,189,5020488,5159285


The first two "columns" (both in `bursts` or `firstburst`) are the index of 
start and stop timestamps (relative to the all-photons timestamps).
The last two columns (`start` and `stop`) are the actual times of burst 
start and stop. To access any of these fields we type:

In [11]:
bursts.istart

array([    153,     184,     215, ..., 2259442, 2259449, 2259492])

In [12]:
firstburst.istart

array([153])

Note that `ph[firstburst.istart]` is equal to `firstburst.start`:

In [13]:
ph[firstburst.istart], firstburst.start

(array([5020488]), array([5020488]))

The same holds for stop:

In [14]:
ph[firstburst.istop], firstburst.stop

(array([5159285]), array([5159285]))

Note that `bursts` is a `Bursts` object (plural) and `firstburst` is a `Burst` object (singular).
You can find more info on these objects in the documentation:

- [Low-level burst search functions](http://fretbursts.readthedocs.org/en/latest/burstsearch.html)

## Burst photon-counts

The variables `nd`, `na`, `naa` contains the number of photon in different photon streams.
By default these values are beckaground corrected and, if the correction coefficients 
are specified, are also corrected for leakage, direct excitation and gamma factor.

To get the raw counts before correction we can redo the burst search as follow:

In [15]:
d.burst_search(computefret=False)
d.calc_fret(count_ph=True, corrections=False)

 - Performing burst search (verbose=False) ...

[DONE]


 - Calculating burst periods ...

[DONE]


Note that if you select bursts, you also need to use `computefret=False`
to avoid recomputing E and S (which by defaults applyies the corrections):

In [16]:
ds = d.select_bursts(select_bursts.size, th1=30, computefret=False)

In [17]:
nd = ds.nd[0]
na = ds.na[0]
naa = ds.naa[0]
E = ds.E[0]
S = ds.S[0]

In [18]:
nd

array([ 31.,   9.,  22., ...,  16.,  14.,   9.])

Note that the burst counts are integer values, confirming that the background
correction was not applied.

# See also

Thhis section of the documentation has more info how to use `Bursts` objects:

- [Low-level burst search functions](http://fretbursts.readthedocs.org/en/latest/burstsearch.html)

In particular, these 3 methods:

- [Bursts.recompute_times](http://fretbursts.readthedocs.org/en/latest/burstsearch.html?highlight=recompute_times#fretbursts.burstsearch.burstsearchlib.Bursts.recompute_times)
- [Bursts.recompute_index_expand](http://fretbursts.readthedocs.org/en/latest/burstsearch.html?highlight=recompute_index_expand#fretbursts.burstsearch.burstsearchlib.Bursts.recompute_index_expand)
- [Bursts.recompute_index_reduce](http://fretbursts.readthedocs.org/en/latest/burstsearch.html?highlight=recompute_index_reduce#fretbursts.burstsearch.burstsearchlib.Bursts.recompute_index_reduce)

Allow to transform burst data to refer a new timestamps arrays.